In [16]:
import psycopg

conn = psycopg.connect("dbname=yelp_dataset host=localhost")
cur = conn.cursor()

query1 = """
    SELECT b.name AS restaurant_name, b.city, COUNT(r.review_id) AS total_reviews
FROM reviews r
JOIN businesses b ON r.business_id = b.business_id
JOIN users u ON r.user_id = u.user_id
WHERE b.categories ILIKE '%Restaurant%'
GROUP BY b.name, b.city
ORDER BY total_reviews DESC
LIMIT 10;
"""
cur.execute(query1)
results = cur.fetchall()

for row in results:
    print(row)

cur.close()
conn.close()


('The Stillery', 'Nashville', 2041)
('Desire Oyster Bar', 'New Orleans', 1098)
('Cold Spring Tavern', 'Santa Barbara', 1036)
("Skull's Rainbow Room", 'Nashville', 755)
('The Twisted Tail', 'Philadelphia', 625)
('Grand Isle', 'New Orleans', 603)
('Maïs Arepas', 'New Orleans', 594)
('Federal Donuts', 'Philadelphia', 567)
('Crazy Sushi', 'Philadelphia', 564)
('Grace Meat + Three', 'St. Louis', 487)


In [14]:
conn = psycopg.connect("dbname=yelp_dataset host=localhost")
cur = conn.cursor()

query1_analyze = """
EXPLAIN ANALYZE
    SELECT b.name AS restaurant_name, b.city, COUNT(r.review_id) AS total_reviews
FROM reviews r
JOIN businesses b ON r.business_id = b.business_id
JOIN users u ON r.user_id = u.user_id
WHERE b.categories ILIKE '%Restaurant%'
GROUP BY b.name, b.city
ORDER BY total_reviews DESC
LIMIT 10;
"""
cur.execute(query1_analyze)
results = cur.fetchall()

for row in results:
    print(row)

cur.close()
conn.close()

('Limit  (cost=13660.80..13660.82 rows=10 width=38) (actual time=210.095..211.442 rows=10 loops=1)',)
('  ->  Sort  (cost=13660.80..13669.66 rows=3544 width=38) (actual time=210.094..211.439 rows=10 loops=1)',)
('        Sort Key: (count(r.review_id)) DESC',)
('        Sort Method: top-N heapsort  Memory: 26kB',)
('        ->  Finalize HashAggregate  (cost=13548.77..13584.21 rows=3544 width=38) (actual time=209.589..211.154 rows=923 loops=1)',)
('              Group Key: b.name, b.city',)
('              Batches: 1  Memory Usage: 241kB',)
('              ->  Gather  (cost=12751.37..13495.61 rows=7088 width=38) (actual time=207.491..209.417 rows=2624 loops=1)',)
('                    Workers Planned: 2',)
('                    Workers Launched: 2',)
('                    ->  Partial HashAggregate  (cost=11751.37..11786.81 rows=3544 width=38) (actual time=204.167..204.391 rows=875 loops=3)',)
('                          Group Key: b.name, b.city',)
('                          Batches: 1 